In [1]:
import os
import pandas as pd
# from bs4 import BeautifulSoup
# import urllib.request
import requests
from IPython.display import display, HTML
import functools

In [2]:
# TFE Rankings
url_tfe = 'https://tfetimes.com/2016-computer-science-rankings/'
df_tfe = pd.read_html(url_tfe,header=0)[0]
df_tfe.drop(['Program'],axis=1,inplace=True)
df_tfe.columns = ["Rank_TFE", "University"]


In [3]:
# US New Rankings
url_usn = 'http://grad-schools.usnews.rankingsandreviews.com/best-graduate-schools/top-science-schools/computer-science-rankings'

url_list = [url_usn]
url_base = 'http://grad-schools.usnews.rankingsandreviews.com/best-graduate-schools/top-science-schools/computer-science-rankings/page+'

for i in range(2,6):
	url_list.append(url_base + str(i))


header = {'User-Agent': 'Mozilla/5.0'} # Needed to prevent 403 error

dfs = []
def read_tables(x):
	for url in x:
		dfs.append(pd.read_html(requests.get(url,headers=header).text,attrs={'class': 'ranking-data table         table-lighthead \
        table-packed ranking-data-free '})[0])

read_tables(url_list)

In [4]:
usn_frames = []
# Cleaning USN data
for df in dfs:
    # Delete row at index 3
    df.drop(df.index[[3]], inplace=True)
    # Rename columns 
    df.columns = ['Rank','University','Score']
    # Strip Rank to extract only number
    df['Rank'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
    df.drop(['Score'],axis=1,inplace=True)
    df.columns = ["Rank_USN", "University"]
    usn_frames.append(df)

In [5]:
df_usn = pd.concat(usn_frames,ignore_index=True)

In [6]:
len(df_usn)

125

In [7]:
# Happy Schools ranking

url_hs = 'http://www.happyschools.com/nrc-computer-science-rankings/'
df_hs = pd.read_html(requests.get(url_hs,headers=header).text,header=0)[0]
df_hs.drop(['R-Rank','S-Rank'],axis=1,inplace=True)
df_hs.columns = ["Rank_HS", "University"]

In [8]:
#df_hs.head(25)
len(df_hs)

128

In [9]:
# msinus ranking

url_msinus = 'http://www.msinus.com/content/computer-science-rankings-us-universities-265/'
df_msinus = pd.read_html(url_msinus,header=None)[0]
df_msinus.columns = ["Rank_MSINUS","University", "Location"]
df_msinus.drop(['Location'],axis=1,inplace=True)
df_msinus.replace('32..','32',inplace=True)

In [10]:
df_msinus = df_msinus.head(80)
df_hs = df_hs.head(120)
df_usn = df_usn.head(120)
df_tfe = df_tfe.head(120)
len(df_usn)

120

In [11]:
df_list = [df_msinus,df_hs,df_usn,df_tfe]

In [12]:
changes1 = {'University of Southern California':'USC'}
changes = {'Berkeley':'UC Berkeley','Carnegie':'CMU','Massachusetts Institute of Technology':'MIT',
          'Stanford':'Stanford University', 'Champaign':'UIUC', 'Cornell':'Cornell University',
           'University of Washington':'UW Seattle','Princeton':'Princeton University', 
           'California Institute Technology':'CalTech','University of Chicago':'UChicago',
           'University of Rochester':'University of Rochester','Stony Brook':'Stony Brook',
           'University of Virginia':'UVa','Indiana University':'IU Bloomington',
           'University of Colorado':'UC Boulder','University Of Pittsburgh':'UPitt',
           'Texas A&m':'TAMU','Texas A &': 'TAMU','Rensselaer':'RPI','University Of Florida':'University Of Florida',
           'Santa Cruz':'UC Santa Cruz','Boston University':'Boston University','Dartmouth College':'Dartmouth',
           'North Carolina State University':'NCSU','Vanderbilt University':'Vanderbilt University',
           'Arizona State University':'ASU','University Of Iowa':'U of Iowa','Iowa State University':'Iowa State University',
           'Buffalo':'SUNY Buffalo','University Of Utah':'University Of Utah',
           'Michigan State University':'Michigan State University',
           'Georgia Institute of Technology':'Georgia Tech', 'Austin':'UT Austin','Madison':'UW Madison', 
           'Los Angeles':'UCLA', 'University of Michigan':'UMich Ann Arbor',
           'Columbia University':'Columbia University', 'College Park':'UMCP','Harvard University':'Harvard University',
           'University of Pennsylvania':'UPenn','Brown University':'Brown University','Rice University':'Rice University',
           'Yale':'Yale University','Duke':'Duke University', 
           'Purdue':'Purdue University','San Diego':'UCSD','Amherst':'UMass Amherst',
           'Chapel Hill':'UNC Chapel Hill','Johns Hopkins':'JHU','New York University':'NYU',
           'Pennsylvania State University':'Penn State','Irvine':'UC Irvine','Minnesota':'University of Minnesota',
           'University of Virgina':'UVa','Northwestern':'NWU','Ohio State':'OSU','Rutgers':'Rutgers','Davis':'UC Davis',
           'Santa Barbara':'UC Santa Barbara','University Of Houston':'University Of Houston',
           'University Of Central Florida':'University Of Central Florida','University Of Nebraska':'University Of Nebraska',
           'George Washington University':'George Washington University','Case Western':'Case Western',
           'Syracuse University':'Syracuse University','Lehigh University':'Lehigh University',
           'Kansas State University':'Kansas State University'
          }

In [13]:
for df in df_list:
    for k,v in changes1.items():
        df.loc[df['University'].str.contains(k, case=False), 'University'] = v    

In [14]:
for df in df_list:
    for k,v in sorted(changes.items()):
        df.loc[df['University'].str.contains(k, case=False), 'University'] = v 

In [17]:
df_final = functools.reduce(lambda left,right: pd.merge(left,right,on='University'), df_list)

In [19]:
df_final = df_final.drop_duplicates(subset = ['University'])

In [20]:
HTML(df_final.to_html())

,Rank_MSINUS,University,Rank_HS,Rank_USN,Rank_TFE
0,1,MIT,2,1,2
1,2,Stanford University,1,1,1
2,3,UC Berkeley,5,1,6
3,4,CMU,4,1,4
4,5,Cornell University,7,6,8
5,6,Princeton University,3,8,3
6,8,UT Austin,14,9,11
7,9,Harvard University,9,18,20
8,10,UIUC,6,5,7
9,11,UW Madison,18,11,13


In [21]:
df_final[['Rank_MSINUS','Rank_USN']] = df_final[['Rank_MSINUS','Rank_USN']].apply(pd.to_numeric)

In [22]:
df_final['Rank_SUM'] = df_final[['Rank_MSINUS','Rank_HS','Rank_TFE','Rank_USN']].sum(axis=1)

In [23]:
result = df_final.sort_values('Rank_SUM')

In [24]:
result.reset_index(inplace=True,drop=True)

In [25]:
HTML(result.to_html())

,Rank_MSINUS,University,Rank_HS,Rank_USN,Rank_TFE,Rank_SUM
0,2,Stanford University,1,1,1,5
1,1,MIT,2,1,2,6
2,4,CMU,4,1,4,13
3,3,UC Berkeley,5,1,6,15
4,6,Princeton University,3,8,3,20
5,5,Cornell University,7,6,8,26
6,10,UIUC,6,5,7,28
7,8,UT Austin,14,9,11,42
8,22,Georgia Tech,12,9,5,48
9,12,UCLA,10,13,15,50
